In [1]:
# 导包和拿数据集
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
iris = load_iris()
logits = iris.data
labels = iris.target
print(logits[:2])
print(labels[:2])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]]
[0 0]


In [2]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(logits , labels , test_size = 0.2)

In [3]:
# 先来看看KNN都有啥参数
'''n_neighbors：KNN中的k值，默认为5
  weights：用于标识每个样本的近邻样本的权重，可选择"uniform",“distance” 或自定义权重。默认"uniform"，
                所有最近邻样本权重都一样。如果是"distance"，则权重和距离成反比例；如果样本的分布是比较成簇的，即各类样本都在相对分开的簇中时，
                我们用默认的"uniform"就可以了，如果样本的分布比较乱，规律不好寻找，选择"distance"是一个比较好的选择.
  n_jobs：并行处理任务数，主要用于多核CPU时的并行处理，加快建立KNN树和预测搜索的速度。n_jobs= -1，即所有的CPU核都参与计算。
  metric，p：距离度量（前面介绍过），默认闵可夫斯基距离 “minkowski”（p=1为曼哈顿距离， p=2为欧式距离）'''

# 开始调参 , 寻找最好的k和p值
%time
best_sorce = 0.0
best_k = -1
best_p = -1
for i in range(1,11):
    for p in range(1,6):
        knn_clf = KNeighborsClassifier(n_neighbors = i , weights = "distance" , p = p)
        knn_clf.fit(X_train , y_train)
        score = knn_clf.score(X_test , y_test)
        if(best_sorce < score):
            best_p = p
            best_sorce = score
            best_k = i
print(best_p)
print(best_sorce)
print(best_k)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
2
0.8666666666666667
1


### 使用sklearn中的GridSearchCV进行调参

In [5]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'weights': ['uniform'], 
        'n_neighbors': [i for i in range(1, 11)]
    },
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(1, 11)], 
        'p': [i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(knn_clf, param_grid , cv=10)
%time
grid_search.fit(X_train, y_train)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=5,
           weights='distance'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [6]:
grid_search.estimator

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=5,
           weights='distance')

In [7]:
# 查看最好的准确率
grid_search.best_score_

0.9916666666666667

In [8]:
# 查看最好的值
grid_search.best_params_

{'n_neighbors': 10, 'p': 2, 'weights': 'distance'}